# Import lib

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import matplotlib.pyplot as plt

# Import data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

DATA_DIR = os.path.join(
    path,
    "asl_alphabet_train",
    "asl_alphabet_train"
)

# Load model

In [ ]:
loaded_model = keras.models.load_model("asl_alphabet_model.h5")
loaded_model.summary()

# Chuẩn bị lớp

In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 128

In [ ]:
# chuan bi label (chu cai)
# cắt tập train : validation + scale ảnh
datagen = ImageDataGenerator(
    rescale=1./255, # chuyen doi gia tri [0, 255] ve [0, 1]
    validation_split=0.2 # chia du lieu thanh 80% train va 20% validation
)
train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # chi dinh lay tap train
)
class_names = train_data.class_indices
class_names = dict((v, k) for k, v in class_names.items())  # dao nguoc key va value
print(class_names)

In [ ]:
from glob import glob
# đi qua tất cả file trong thư mục test
test_images = glob(os.path.join(path,  "asl_alphabet_train", "asl_alphabet_train", "nothing/**"))
test_images[10:15]  # in ra 5 file đầu tiên

# Dự đoán

In [ ]:
%matplotlib inline

In [ ]:
# ve hinh anh 
import cv2
test_img_path = test_images[12]
img = cv2.imread(test_img_path)
plt.rcParams['figure.figsize'] = [5, 5]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title(test_img_path.split("\\")[-1])
plt.axis('off')
plt.show()

In [ ]:
# du doan hinh
img = image.load_img(test_img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # scale anh
img_array = np.expand_dims(img_array, axis=0) 

In [ ]:
# du doan hih
prediction = loaded_model.predict(img_array)
predicted_probs = tf.nn.softmax(prediction[0])

# Get top 2 predictions
top_2_indices = np.argsort(predicted_probs)[-2:]
top_1_prob = predicted_probs[top_2_indices[1]]
top_2_prob = predicted_probs[top_2_indices[0]]

PROBABILITY_GAP = 0.3  # top prediction should be at least 30% higher

if top_1_prob < 0.6 or (top_1_prob - top_2_prob) < PROBABILITY_GAP:
    print("ERROR: Ambiguous or invalid image!")
    print(f"Top prediction: {top_1_prob:.2%}, Second: {top_2_prob:.2%}")
else:
    predicted_label = class_names[top_2_indices[1]]
    print(f'Predicted label: {predicted_label} (Confidence: {top_1_prob:.2%})')